In [1]:
import gym
import gym_kraby
import math as maths
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from copy import deepcopy as DC
import torch
import time
clear_output()

In [2]:
class hexapod:
    def __init__(self,render=False):
        self.env= gym.make('gym_kraby:HexapodBulletEnv-v0', render=render)
        obs=self.env.reset()
        self.action_space=self.env.action_space.sample()  # take a random action
        self.sensors={"foot_0":4,"endcap_0":5,"foot_1":10,"endcap_1":11,
            "foot_2":16,"endcap_2": 17,"foot_3": 22,"endcap_3": 23,
            "foot_4": 28,"endcap_4": 29,"foot_5": 34,"endcap_5":35}
    def list_feet(self):
        _link_name_to_index = {self.env.p.getBodyInfo(self.env.robot_id)[0].decode('UTF-8'):-1,}
        for _id in range(self.env.p.getNumJoints(self.env.robot_id)):
            _name = self.env.p.getJointInfo(self.env.robot_id, _id)[12].decode('UTF-8')
            _link_name_to_index[_name] = _id
            if "endcap" in _name or "foot" in _name:
                print(_name,_id)
    def reset(self):
        obs=self.env.reset()
    def step(self,a):
        """
        step through environment with action a
        return reward and the force on the feet that are contacted numbered 0-5 for each leg
        """
        observation, reward, done, _ = self.env.step(a)  # step
        filtered=self.getContact()
        return reward,filtered
    def getAngles(self):
        a=[self.getAngle(i) for i in range(18)]
        return np.array(a)
    def getContact(self):
        VALS=np.array([[5,4],[11,10],[17,16],[23,22],[29,28],[35,34]])
        contact=self.env.p.getContactPoints(bodyA=self.env.robot_id)
        a_=np.array([[5,0],[11,0],[17,0],[23,0],[29,0],[35,0]])
        for c in contact:
            p=c[3]
            f=c[9]
            if p in VALS.flatten():
                id=np.where(VALS == p)
                id=[id[0],id[1]][0]
                if a_[id[0]][0]==p: a_[id[0]][1]=f
                elif a_[id[0]][1]<f: #largest force saves
                    a_[id[0]][0]=p
                    a_[id[0]][1]=f
        return a_
    def getAngle(self,num):
        JOINT=[1,2,4,7,8,10,13,14,16,19,20,22,25,26,28,31,32,34]
        b=self.env.p.getJointState(self.env.robot_id,JOINT[num])
        d=b[0]#convert to degrees
        return (maths.pi/2)+d
    def getImage(self):
        img = self.env.p.getCameraImage(224, 224, renderer=self.env.p.ER_BULLET_HARDWARE_OPENGL)
        return img[2]
    def close(self):
        self.env.close()
    def moveAngles(self,ang):
        for j in range(5): #for accuracy do multiple times
            mov=np.zeros_like(ang)
            angles=self.getAngles()
            for i in range(len(mov)):
                mov[i]=self.moveToAngle(ang[i],angles[i])
            self.step(mov)
    def moveToAngle(self,start,to):
        end=start-to
        k=1/np.deg2rad(18)
        return end*k
    def place(self,legNo,ANG1,ANG2=None):
        legNo=legNo*3
        act=self.getAngles()
        if np.around(act[legNo+1],2)!=np.around(ANG1,2): #don't replace if same
            act[legNo+1]=ANG1
            ang=(np.pi)-ANG1
            act[legNo+2]=ang
        if ANG2!=None and np.around(ANG2,2)!=np.around(act[legNo],2): act[legNo]=ANG2
        self.moveAngles(act)
    def moveThrough(self,ANGLES):
        for i in range(6):
            self.place(i,ANGLES[:,0][i],ANG2=ANGLES[:,1][i])
    

In [3]:
robot=hexapod()
act=np.ones_like(robot.action_space)*np.deg2rad(80)
robot.moveAngles(act)
robot.place(5,np.deg2rad(120))


c:\Users\dexte\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\dexte\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\dexte\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\dexte\AppData\Local\Programs\Python\Pyt

In [4]:
def place(legNo,ANG1,ANG2=None):
        legNo=legNo*3
        act=robot.getAngles()
        act[legNo+1]=ANG1
        ang=(np.pi)-ANG1
        act[legNo+2]=ang
        if ANG2!=None: act[legNo]=ANG2
        robot.moveAngles(act)
place(1,np.deg2rad(180))

## Evolving patterns

### Model

In [5]:

class CPG:
    def __init__(self, num_neurons, layers, coupling_strength):
        self.num_neurons = num_neurons
        self.layers=layers
        self.coupling_strength = coupling_strength
        self.phase_offset = np.random.uniform(-np.pi, np.pi, size=num_neurons)
        self.frequency = np.random.uniform(0.5, 2.0, size=num_neurons)
        #self.amplitude = np.random.uniform(0.5, 2.0, size=num_neurons)
        self.bias = np.random.uniform(-1.0, 1.0, size=num_neurons)
        self.phase = np.zeros((num_neurons,layers))
        self.output = np.zeros((num_neurons,layers))
        self.weights = np.random.uniform(-1.0, 1.0, size=(num_neurons, num_neurons))
        self.num_genes = (self.weights.flatten().shape[0]+
                          self.bias.flatten().shape[0]+
                          self.frequency.flatten().shape[0])
    def step(self, inputs, dt):
        inputs=inputs.reshape(self.num_neurons,1)
        # Calculate the coupling between neurons using the evolved weights
        coupling = self.coupling_strength * np.dot(self.output.T,self.weights)
        
        # Update the phase of each neuron
        self.phase += np.add((2 * np.pi * self.frequency * dt).reshape(self.num_neurons,1),
                                coupling.reshape(self.num_neurons,self.layers))
        # Calculate the output of each neuron
        self.output = np.sin(self.phase ) + self.bias.reshape(self.num_neurons,1)
        
        # Add inputs from sensors to the output
        self.output = np.add(self.output ,inputs)
    def reset(self):
        self.phase = np.zeros((self.num_neurons,self.layers))
        self.output = np.zeros((self.num_neurons,self.layers))
    def set_genes(self,genotype):
        self.weights=genotype[0:self.weights.flatten().shape[0]].reshape(self.num_neurons,self.num_neurons)
        self.bias=genotype[self.weights.flatten().shape[0]:self.weights.flatten().shape[0]+self.bias.flatten().shape[0]]
        self.frequency=genotype[self.weights.flatten().shape[0]+self.bias.flatten().shape[0]:]
    def forward(self,inputs,dt):
        self.step(inputs,dt)
        instruct=np.tanh(self.output)
        instruct[instruct<0]=-1
        instruct[instruct>=0]=1
        return instruct
cpg=CPG(6,2,2)
cpg.set_genes(np.random.normal(0,1,(cpg.num_genes,)))
x=np.random.normal(0,1,(6,))
cpg.forward(x,0.1)


array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [-1., -1.],
       [-1., -1.],
       [ 1.,  1.]])

### GA

In [10]:
def mutate(geno):
    geno+=np.random.normal(0,3,(geno.shape))
    geno[geno>3]=3
    geno[geno<-3]=-3
    return 

def gen_genotype(shape): #generate a genotype of 1 and 0
    return np.random.normal(0,5,(shape))

def gen_pop(size,pergeno):
    return gen_genotype(size*pergeno).reshape((size,pergeno))

def fitness(robot, start, energy_consumption):
    # Define weights for each fitness component
    cubePos, end_orientation = robot.env.p.getBasePositionAndOrientation(robot.env.robot_id)
    distance_travelled_x=np.abs(np.array(start)[0]-np.array(cubePos)[0])
    distance_travelled_y=np.abs(np.array(start)[1]-np.array(cubePos)[1])

    if np.inf == fitness:
        return 0
    return max(distance_travelled_x-distance_travelled_y,0)
def getFallen():
    cubePos, cubeOrn = robot.env.p.getBasePositionAndOrientation(robot.env.robot_id)
    if cubePos[-1]<0: return True
    return False
def runTrial(agent,robot,T):
    dt=0.02
    robot.reset()
    agent.reset()
    done=False
    t=0
    fx=0
    pattern=np.zeros((12))
    last=0
    prev_pattern=0
    start, cubeOrn = robot.env.p.getBasePositionAndOrientation(robot.env.robot_id)
    while t<T and not done:
        """r,c=robot.step(pattern)
        f=c[:,1] #get forces"""
        #angles=robot.getAngles().reshape(6,3)
        start, cubeOrn = robot.env.p.getBasePositionAndOrientation(robot.env.robot_id)
        forces=robot.getContact()[:,1]
        pattern=agent.forward(forces,dt)
        angles=pattern.reshape(6,2)
        #convert to standing
        angles[:,0][angles[:,0]>0]=np.deg2rad(120)
        angles[:,0][angles[:,0]<=0]=np.deg2rad(80)
        #convert to swing
        angles[:,1][angles[:,1]>0]=np.deg2rad(120)
        angles[:,1][angles[:,1]<=0]=np.deg2rad(40)
        robot.moveThrough(angles) #move robot
        #fx+=np.sum(np.abs(pattern))/(len(pattern)*10)
        f_=fitness(robot,start,fx)
        if getFallen():
            done=True
        if f_>last:
            fx+=1
        last=f_
        t+=dt
    
    #f=fitness(robot,start
    return fx/(T/dt)
def run(agent,robot,population,generations=500,T=2,gen=0):
    pop_size=len(population)
    shape=population.shape[1]
    fitness_matrix=np.zeros((pop_size))
    overTime=np.zeros((generations,))
    catch=-1
    while gen < (generations) and overTime[max(gen-1,0)]<1:
        clear_output()
        print("Generation:",gen,"Fitness",overTime[max(gen-1,0)])
        #get mask to select genotypes for battle
        mask=np.random.choice([0, 1], size=pop_size)
        inds=(mask==1).nonzero()[0]
        while len(inds)%2!=0:
            mask=np.random.choice([0, 1], size=pop_size,p=[0.8, 0.2])
            inds=(mask==1).nonzero()[0]
        #get indicies and tournament modes
        inds=inds.reshape(len(inds)//2,2).astype(int)
        fitnesses=np.zeros((len(inds)//2)).astype(int)
        new_inds=np.zeros((len(inds)//2,2)).astype(int)
        #run each trial
        for i in range(len(inds)//2):
            #select genotypes
            g1=population[inds[i][0]]
            g2=population[inds[i][1]]
            #tournament
            agent.set_genes(g1)
            f1=0
            for j in range(3):
                f1+=runTrial(agent,robot,T)
                if getFallen(): break
            f1/=3
            agent.set_genes(g1)
            f2=0
            if not getFallen():
                for j in range(3):
                    if getFallen(): break
                    f2+=runTrial(agent,robot,T)
                f2/=3
            fitness_matrix[inds[i][0]]=f1
            fitness_matrix[inds[i][1]]=f2
            if f1>f2:
                fitnesses[i]=f1
                new_inds[i]=[0,i]
            elif f2>f1:
                fitnesses[i]=f2
                new_inds[i]=[1,i]
            else:
                fitnesses[i]=0
                new_inds[i]=[0,i]
            if getFallen():
                break
        if getFallen():
            np.save("model_cpg_noRendA",population)
            print("Reset")
            break
        else:
            #get top values and redistribute into the array
            winners=int(len(inds)//2 *0.4)
            mutants=int(len(inds)//2 *0.4)
            other=len(inds)//2 -winners - mutants
            order=np.argsort(fitnesses)
            for i in reversed(range(len(inds)//2)): #loop through backwards leaving the winners in place
                genoWin=new_inds[i][0]
                old_index=new_inds[i][1]
                if i<(len(inds)//2)-winners and i>(len(inds)//2)-winners-mutants: #pick mutants
                    population[inds[old_index][1-genoWin]]=mutate(DC(population[inds[old_index][genoWin]])) #mutate copy
                elif i<(len(inds)//2)-winners-mutants: #the other
                    population[inds[old_index][1-genoWin]]=gen_genotype(shape=shape) #create new genotype
        overTime[gen]=np.max(fitness_matrix)
        gen+=1
    """except KeyboardInterrupt:
        np.save("model_cpg_noRendA",population)"""
    return overTime,fitness_matrix,gen


In [ ]:
gens=np.zeros((500,))

In [11]:
agent=CPG(6,2,2)
population=gen_pop(50,agent.num_genes) #create population
try: population=np.load("model_cpg_noRendA") #if exists
except: pass

agent.set_genes(population[0])
g=132

while g<200:
    try:
        t,f,g=run(agent,robot,population,generations=500,T=1,gen=g)
        robot.close()
        robot=hexapod()
        agent=CPG(6,2,2)
        agent.set_genes(population[0])
        time.sleep(2)
        np.save("model_cpg_noRendA",population)
        gens=gens+t
    except KeyboardInterrupt:
        np.save("model_cpg_noRendA",population)
        break


Generation: 133 Fitness 0.49333333333333335


In [ ]:
robot.close()
robot=hexapod()
agent=CPG(6,2,2)
agent.set_genes(population[0])
runTrial(agent,robot,2)

0.37

In [ ]:

def microbial(agent,robot,population,generations=500,T=2):
    pop_size=len(population)
    overTime=np.zeros((generations,))
    fitness_matrix=np.zeros((pop_size))
    for gen in range(34,generations): #run trial
        clear_output()
        print("Generation:",gen,"Fitness",overTime[max(gen-1,0)])
        n1=np.random.randint(0,pop_size-1)
        n2=n1+1
        if n2==pop_size: n2=n1-1
        #while n1==n2: n2=np.random.randint(0,pop_size-1) #make sure it is not the same
        g1=DC(population[n1])
        g2=DC(population[n2])
        #get fitnesses
        agent.set_genes(g1)
        f1=0
        for j in range(3):
            r=runTrial(agent,robot,T)
            f1+=r
        f1/=3
        agent.set_genes(g1)
        f2=0
        for j in range(3):
            r=runTrial(agent,robot,T)
            f2+=r
        f2/=3
        fitness_matrix[n1]=f1
        fitness_matrix[n2]=f2
        
        if getFallen():
            print("Reset")
            time.sleep(10)
            robot.close()
            agent=CPG(6,2,2)
            robot=hexapod()
        #tournament
        if f1>f2:
            population[n2]=DC(mutate(DC(g1)))
        elif f2>f1:
            population[n1]=DC(mutate(DC(g2)))
        overTime[gen]=np.max(fitness_matrix)
    """except KeyboardInterrupt:
        np.save("model_cpg_noRend",population)"""
    return overTime,fitness_matrix